In [63]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [40]:
df = pd.read_csv("/Users/guillermocomesanacimadevila/Desktop/PhD/GenomicSEM/Part1/Checks/HDLL_per_block/local_rg_per_block.tsv",sep="\t")

In [47]:
df.columns
df[["Heritability_1", "Heritability_2"]] = df[["Heritability_1", "Heritability_2"]].apply(pd.to_numeric, errors='coerce')

In [50]:
print(f"SZ h2 = {df["Heritability_1"].sum()}")
print(f"AD h2 = {df["Heritability_2"].sum()}")
print(f"Covg = {df["Genetic_Covariance"].sum()}")
rg_global = df["Genetic_Covariance"].sum() / np.sqrt(
    df["Heritability_1"].sum() * df["Heritability_2"].sum()
)
print(f"rg = {rg_global}")

# SZ h2 = 0.19047911746545068
# AD h2 = 0.3440573496199979

SZ h2 = 0.19047911746545068
AD h2 = 0.3440573496199979
Covg = 0.004377972766161657
rg = 0.01710149000024233


In [57]:
df = df.replace([np.inf, -np.inf], np.nan)

In [58]:
h2_scz = np.clip(df["Heritability_1"].to_numpy(dtype=float), 0, None).sum()
h2_ad  = np.clip(df["Heritability_2"].to_numpy(dtype=float), 0, None).sum()
gcov   = df["Genetic_Covariance"].fillna(0.0).to_numpy(dtype=float).sum()

rg_global = gcov / np.sqrt(h2_scz * h2_ad)
print(f"SCZ h² = {h2_scz:.4f} ({h2_scz*100:.2f}%)")
print(f"AD  h² = {h2_ad:.4f} ({h2_ad*100:.2f}%)")
print(f"Cov_g = {gcov:.6f}")
print(f"r_g   = {rg_global:.3f}")

SCZ h² = 0.1905 (19.05%)
AD  h² = 0.3441 (34.41%)
Cov_g = 0.004378
r_g   = 0.017


In [65]:
print(df.columns)

Index(['CHR', 'piece', 'Trait1', 'Trait2', 'eigen_use', 'Heritability_1',
       'P_value_Heritability_1', 'Heritability_2', 'P_value_Heritability_2',
       'Genetic_Covariance', 'Genetic_Correlation', 'Lower_bound_rg',
       'Upper_bound_rg', 'P', 'n_ref', 'cov_scz', 'cov_ad', 'bp_min', 'bp_max',
       'local_rg'],
      dtype='object')


In [71]:
len([i for i in df["Genetic_Correlation"] if i == 0 or pd.isna(i)])

1669

In [74]:
len(df["Genetic_Correlation"].tolist())

2459

In [77]:
num_cols = ["Heritability_1", "Heritability_2", "Genetic_Covariance"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")
df = df.replace([np.inf, -np.inf], np.nan)

h2_scz_raw = np.nan_to_num(df["Heritability_1"].to_numpy(float), nan=0.0).sum()
h2_ad_raw  = np.nan_to_num(df["Heritability_2"].to_numpy(float), nan=0.0).sum()
gcov_raw   = np.nan_to_num(df["Genetic_Covariance"].to_numpy(float), nan=0.0).sum()

print(f"[RAW] SCZ h² = {h2_scz_raw:.6f}")
print(f"[RAW]  AD h² = {h2_ad_raw:.6f}")
print(f"[RAW] Cov_g  = {gcov_raw:.6f}")

for label, col in [("SCZ", "Heritability_1"), ("AD", "Heritability_2")]:
    x = df[col].dropna()
    n_neg = (x < 0).sum()
    sum_neg = x[x < 0].sum()
    p99 = x.quantile(0.99)
    p997 = x.quantile(0.997)
    top5 = x.nlargest(5).values
    print(f"{label}: n_blocks={x.size}, negatives={n_neg} (sum={sum_neg:.6f}), "
          f"p99={p99:.6g}, p99.7={p997:.6g}, top5={np.array2string(top5, precision=6)}")

def winsorize(s, q=0.999):
    cap = s.quantile(q)
    return np.where(s.notna(), np.minimum(s, cap), np.nan)

df["h2_scz_w"] = winsorize(df["Heritability_1"], q=0.999)
df["h2_ad_w"]  = winsorize(df["Heritability_2"],  q=0.999)

h2_scz_win = np.nan_to_num(df["h2_scz_w"], nan=0.0).sum()
h2_ad_win  = np.nan_to_num(df["h2_ad_w"],  nan=0.0).sum()

rg = gcov_raw / np.sqrt(max(h2_scz_raw, 0) * max(h2_ad_raw, 0)) if h2_scz_raw>0 and h2_ad_raw>0 else np.nan

print(f"[WINSORIZED] SCZ h² = {h2_scz_win:.6f}")
print(f"[WINSORIZED]  AD h² = {h2_ad_win:.6f}")
print(f"r_g (from raw sums) = {rg:.6f}")

[RAW] SCZ h² = 0.190479
[RAW]  AD h² = 0.344057
[RAW] Cov_g  = 0.004378
SCZ: n_blocks=2459, negatives=0 (sum=0.000000), p99=0.000515654, p99.7=0.000726644, top5=[0.0015   0.000944 0.000845 0.000833 0.000812]
AD: n_blocks=2459, negatives=0 (sum=0.000000), p99=0.00021289, p99.7=0.000308157, top5=[0.282728 0.001114 0.000692 0.000685 0.000389]
[WINSORIZED] SCZ h² = 0.189709
[WINSORIZED]  AD h² = 0.061590
r_g (from raw sums) = 0.017101
